In [1]:
import tkinter as tk
from tkinter import messagebox
import csv
import logging

In [2]:
# Configure logging for clear, minimal warnings
logging.basicConfig(level=logging.WARNING, format='%(levelname)s:%(message)s')

def load_data(csv_path="data.csv"):
    users = []
    feedback_data = []
    report_data = []

    with open(csv_path, mode="r") as file:
        reader = csv.DictReader(file)
        for row in reader:
            row_type = row.get("type")
            if row_type == "user":
                users.append({
                    "name": row["username"],
                    "password": row["password"],
                    "role": row["role"]
                })
            elif row_type == "feedback":
                feedback_data.append({
                    "feedback_id": row["feedback_id"],
                    "agent": row["agent"],
                    "feedback_text": row["feedback_text"],
                    "status": row["status"]
                })
            elif row_type == "report":
                report_data.append({
                    "report_id": row["report_id"],
                    "date": row["date"],
                    "agent": row["agent"],
                    "feedback_applied": row["feedback_applied"]
                })

    return users, feedback_data, report_data

In [3]:
def save_data(users, feedback_data, report_data, csv_path="data.csv"):
    with open(csv_path, mode="w", newline="") as file:
        fieldnames = ["type", "username", "password", "role", "feedback_id", "agent", "feedback_text", "status", "report_id", "date", "feedback_applied"]
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()

        # Save users
        for user in users:
            writer.writerow({
                "type": "user",
                "username": user["name"],
                "password": user["password"],
                "role": user["role"]
            })

        # Save feedback data
        for feedback in feedback_data:
            writer.writerow({
                "type": "feedback",
                "feedback_id": feedback["feedback_id"],
                "agent": feedback["agent"],
                "feedback_text": feedback["feedback_text"],
                "status": feedback["status"]
            })

        # Save report data
        for report in report_data:
            writer.writerow({
                "type": "report",
                "report_id": report["report_id"],
                "date": report["date"],
                "agent": report["agent"],
                "feedback_applied": report["feedback_applied"]
            })

In [4]:
class App(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Agent and Coach App")
        self.geometry("400x300")

        # Initialize current_user as None
        self.current_user = None

        # Load data once and store it in attributes
        self.users, self.feedback_data, self.report_data = load_data()

        # Container frame to hold all screens
        container = tk.Frame(self)
        container.pack(fill="both", expand=True)
        
        # Dictionary to store frames
        self.frames = {}

        # Add each screen frame to the dictionary
        for F in (LoginScreen, AgentMainMenu, CoachMainMenu, ProfileSettings, ViewFeedback, AgentViewFeedback):
            frame = F(container, self)
            self.frames[F] = frame
            frame.grid(row=0, column=0, sticky="nsew")

        # Start with the login screen after frames are fully initialized
        self.show_frame(LoginScreen)

    def show_frame(self, frame_class):
        """Bring the specified frame to the front of the GUI."""
        frame = self.frames[frame_class]
        
        # Check if we're showing AgentViewFeedback and refresh feedback on login
        if frame_class == AgentViewFeedback:
            frame.refresh_feedback_on_login()
        
        frame.tkraise()

In [5]:
class LoginScreen(tk.Frame):
    def __init__(self, parent, controller):
        super().__init__(parent)
        self.controller = controller

        tk.Label(self, text="Login Screen").pack(pady=10)
        
        # Role selection
        self.role = tk.StringVar(value="Agent")
        tk.Radiobutton(self, text="Agent", variable=self.role, value="Agent").pack()
        tk.Radiobutton(self, text="Coach", variable=self.role, value="Coach").pack()
        
        # Username and Password
        tk.Label(self, text="Username").pack()
        self.username_entry = tk.Entry(self)
        self.username_entry.pack()

        tk.Label(self, text="Password").pack()
        self.password_entry = tk.Entry(self, show="*")
        self.password_entry.pack()
        
        # Login button
        login_button = tk.Button(self, text="Login", command=self.login)
        login_button.pack()

    def login(self):
        username = self.username_entry.get()
        password = self.password_entry.get()
        role = self.role.get()

        # Search for the user in the list
        user_found = None
        for user in self.controller.users:
            if user["name"] == username and user["password"] == password and user["role"] == role:
                user_found = user
                break

        # Validate the found user
        if user_found:
            # Set current_user to the full user dictionary upon successful login
            self.controller.current_user = user_found
            if role == "Agent":
                self.controller.show_frame(AgentMainMenu)
            elif role == "Coach":
                self.controller.show_frame(CoachMainMenu)
        else:
            messagebox.showerror("Login Error", "Username not found or incorrect credentials.")

In [6]:
class AgentMainMenu(tk.Frame):
    def __init__(self, parent, controller):
        super().__init__(parent)
        self.controller = controller

        tk.Label(self, text="Agent Main Menu").pack(pady=10)
        
        # Profile and View Feedback buttons
        profile_button = tk.Button(self, text="Profile", command=lambda: controller.show_frame(ProfileSettings))
        profile_button.pack()
        
        # Feedback buttons
        view_feedback_button = tk.Button(self, text="View Feedback", command=lambda: controller.show_frame(AgentViewFeedback))
        view_feedback_button.pack()

        # Logout button
        logout_button = tk.Button(self, text="Logout", command=self.logout)
        logout_button.pack()

    def logout(self):
        """Clear the current user and go back to the login screen."""
        self.controller.current_user = None  # Reset current_user
        self.controller.show_frame(LoginScreen)

In [7]:
class CoachMainMenu(tk.Frame):
    def __init__(self, parent, controller):
        super().__init__(parent)
        self.controller = controller

        tk.Label(self, text="Coach Main Menu").pack(pady=10)
        
        # View Profile button
        profile_button = tk.Button(self, text="View Profile", command=lambda: controller.show_frame(ProfileSettings))
        profile_button.pack(pady=5)
        
        # Create Account
        create_account_button = tk.Button(self, text="Create Account", command=self.open_create_account_form)
        create_account_button.pack()
        
        # Update Feedback button to navigate to ViewFeedback
        update_feedback_button = tk.Button(self, text="Update Feedback", command=lambda: controller.show_frame(ViewFeedback))
        update_feedback_button.pack(pady=5)
        
        # Input Feedback
        input_feedback_button = tk.Button(self, text="Input Feedback", command=self.open_feedback_input_form)
        input_feedback_button.pack()

        # Generate Reports buttons
        generate_reports_button = tk.Button(self, text="Generate Reports", command=self.generate_reports)
        generate_reports_button.pack()

        logout_button = tk.Button(self, text="Logout", command=lambda: controller.show_frame(LoginScreen))
        logout_button.pack()

    def open_create_account_form(self):
        # Open the account creation form in a new window
        create_account_window = tk.Toplevel(self)
        create_account_window.title("Create New Account")
        
        # Username input
        tk.Label(create_account_window, text="Username").grid(row=0, column=0, padx=10, pady=5)
        username_entry = tk.Entry(create_account_window)
        username_entry.grid(row=0, column=1, padx=10, pady=5)

        # Password input
        tk.Label(create_account_window, text="Password").grid(row=1, column=0, padx=10, pady=5)
        password_entry = tk.Entry(create_account_window, show="*")
        password_entry.grid(row=1, column=1, padx=10, pady=5)

        # Role selection (default to "Agent")
        tk.Label(create_account_window, text="Role").grid(row=2, column=0, padx=10, pady=5)
        role_var = tk.StringVar(value="Agent")
        role_menu = tk.OptionMenu(create_account_window, role_var, "Agent", "Coach")
        role_menu.grid(row=2, column=1, padx=10, pady=5)

        # Submit button
        submit_button = tk.Button(
            create_account_window, 
            text="Create Account", 
            command=lambda: self.create_account(username_entry.get(), password_entry.get(), role_var.get(), create_account_window)
        )
        submit_button.grid(row=3, columnspan=2, pady=10)

    def create_account(self, username, password, role, window):
        # Check for empty fields
        if not username or not password:
            messagebox.showerror("Input Error", "Username and password cannot be empty.")
            return

        # Check if username already exists
        if any(user['name'] == username for user in self.controller.users):
            messagebox.showerror("Input Error", "Username already exists.")
            return

        # Add new user to the users list
        self.controller.users.append({"name": username, "password": password, "role": role})

        # Save changes to the CSV file
        save_data(self.controller.users, self.controller.feedback_data, self.controller.report_data)

        # Close the account creation form
        window.destroy()
        messagebox.showinfo("Account Created", f"Account for {username} created successfully.")
    
    # Adding new feedback for selected agent
    def open_feedback_input_form(self):
        # Open a new window for feedback input
        feedback_window = tk.Toplevel(self)
        feedback_window.title("Input New Feedback")
        
        # Agent selection dropdown
        tk.Label(feedback_window, text="Agent").grid(row=0, column=0, padx=10, pady=5)
        agent_names = [user['name'] for user in self.controller.users if user['role'] == 'Agent']
        agent_var = tk.StringVar(value="Select Agent")
        agent_dropdown = tk.OptionMenu(feedback_window, agent_var, *agent_names)
        agent_dropdown.grid(row=0, column=1, padx=10, pady=5)

        # Feedback text entry
        tk.Label(feedback_window, text="Feedback Text").grid(row=1, column=0, padx=10, pady=5)
        feedback_text_entry = tk.Entry(feedback_window, width=40)
        feedback_text_entry.grid(row=1, column=1, padx=10, pady=5)

        # Submit button
        submit_button = tk.Button(
            feedback_window, 
            text="Submit Feedback", 
            command=lambda: self.add_feedback(agent_var.get(), feedback_text_entry.get(), feedback_window)
        )
        submit_button.grid(row=2, columnspan=2, pady=10)

    def add_feedback(self, agent, feedback_text, window):
        # Check for empty fields
        if not agent or agent == "Select Agent" or not feedback_text:
            messagebox.showerror("Input Error", "Please complete all fields.")
            return

        # Create a new feedback entry with status set to "Not Integrated"
        new_feedback = {
            "feedback_id": f"feedback_{len(self.controller.feedback_data) + 1}",  # Unique ID
            "agent": agent,
            "feedback_text": feedback_text,
            "status": "Not Integrated"  # Default status for new feedback
        }

        # Add the feedback to the feedback_data list
        self.controller.feedback_data.append(new_feedback)

        # Save to CSV
        save_data(self.controller.users, self.controller.feedback_data, self.controller.report_data)

        # Close the feedback input window
        window.destroy()
        messagebox.showinfo("Success", "Feedback added successfully.")

    def generate_reports(self):
        # Placeholder for Generate Reports functionality
        messagebox.showinfo("Generate Reports", "Generate Reports screen (not implemented).")
        
    def logout(self):
        """Clear the current user and go back to the login screen."""
        self.controller.current_user = None  # Reset current_user
        self.controller.show_frame(LoginScreen)

In [8]:
class ProfileSettings(tk.Frame):
    def __init__(self, parent, controller):
        super().__init__(parent)
        self.controller = controller

        tk.Label(self, text="Profile Settings").pack(pady=10)

        # Password fields
        tk.Label(self, text="Current Password").pack()
        self.current_password = tk.Entry(self, show="*")
        self.current_password.pack()

        tk.Label(self, text="New Password").pack()
        self.new_password = tk.Entry(self, show="*")
        self.new_password.pack()

        # Save button
        save_button = tk.Button(self, text="Save Changes", command=self.update_password)
        save_button.pack(pady=5)

        # Back button to return to the main menu
        back_button = tk.Button(self, text="Back to Main Menu", command=self.go_back)
        back_button.pack(pady=5)

    def update_password(self):
        current_pw = self.current_password.get()
        new_pw = self.new_password.get()
        
        # Ensure current_user is a dictionary and has the required attributes
        if not self.controller.current_user:
            messagebox.showerror("Error", "No current user logged in.")
            return

        # Validate current password and update if it matches
        user = self.controller.current_user
        if user["password"] == current_pw:
            user["password"] = new_pw
            try:
                save_data(self.controller.users, self.controller.feedback_data, self.controller.report_data)
                messagebox.showinfo("Profile", "Password updated successfully.")
            except Exception as e:
                messagebox.showerror("Error", f"Failed to save changes: {e}")
        else:
            messagebox.showerror("Error", "Current password is incorrect.")

    def go_back(self):
        """Navigate back to the correct main menu based on the current user's role."""
        # Access the role directly from current_user
        user_role = self.controller.current_user.get("role")

        # Navigate to the correct frame based on the role
        if user_role == "Coach":
            self.controller.show_frame(CoachMainMenu)
        elif user_role == "Agent":
            self.controller.show_frame(AgentMainMenu)
        else:
            messagebox.showerror("Navigation Error", "Unable to determine user role for navigation.")

In [9]:
class ViewFeedback(tk.Frame):
    def __init__(self, parent, controller):
        super().__init__(parent)
        self.controller = controller

        tk.Label(self, text="Coach: View and Update Feedback").pack(pady=10)
        
        # Agent selection dropdown
        tk.Label(self, text="Select Agent").pack(pady=5)
        self.agent_var = tk.StringVar(value="Select Agent")
        agent_names = [agent['name'] for agent in self.controller.users if agent['role'] == 'Agent']
        self.agent_dropdown = tk.OptionMenu(self, self.agent_var, *agent_names, command=self.filter_feedback_by_agent)
        self.agent_dropdown.pack()

        # Listbox to display feedback
        self.feedback_listbox = tk.Listbox(self, width=50, height=10)
        self.feedback_listbox.pack(pady=5)
        self.feedback_listbox.bind("<<ListboxSelect>>", self.on_feedback_select)

        # Status dropdown to assign feedback status
        tk.Label(self, text="Assign Status").pack(pady=5)
        self.status_var = tk.StringVar(value="Not Integrated")
        self.status_dropdown = tk.OptionMenu(self, self.status_var, "Not Integrated", "Integrated")
        self.status_dropdown.pack()

        # Save button to update feedback status
        save_button = tk.Button(self, text="Save Status", command=self.update_feedback_status)
        save_button.pack(pady=10)

        # Back button to return to the main menu
        back_button = tk.Button(self, text="Back to Main Menu", command=lambda: controller.show_frame(CoachMainMenu))
        back_button.pack(pady=5)

    def filter_feedback_by_agent(self, agent_name):
        """Filter feedback based on selected agent."""
        feedback_data = [fb for fb in self.controller.feedback_data if fb["agent"] == agent_name]
        self.populate_feedback_list(feedback_data)

    def populate_feedback_list(self, feedback_data):
        """Display feedback entries for the selected agent in the listbox."""
        self.feedback_listbox.delete(0, tk.END)
        for feedback in feedback_data:
            feedback_text = f"{feedback['feedback_id']}: {feedback['feedback_text']} (Status: {feedback['status']})"
            self.feedback_listbox.insert(tk.END, feedback_text)

    def on_feedback_select(self, event):
        """Handle selection of feedback in the listbox."""
        try:
            selected_index = self.feedback_listbox.curselection()[0]
            selected_feedback = self.controller.feedback_data[selected_index]
            self.status_var.set(selected_feedback["status"])
        except IndexError:
            pass

    def update_feedback_status(self):
        """Update the selected feedback's status and save it."""
        try:
            selected_index = self.feedback_listbox.curselection()[0]
            selected_feedback = self.controller.feedback_data[selected_index]
            selected_feedback["status"] = self.status_var.get()
            save_data(self.controller.users, self.controller.feedback_data, self.controller.report_data)
            messagebox.showinfo("Success", "Feedback status updated successfully.")
            self.populate_feedback_list(self.controller.feedback_data)
        except IndexError:
            messagebox.showerror("Selection Error", "Please select a feedback entry to update.")

In [10]:
class AgentViewFeedback(tk.Frame):
    def __init__(self, parent, controller):
        super().__init__(parent)
        self.controller = controller

        tk.Label(self, text="Agent: View Feedback").pack(pady=10)
        
        # Dropdown to filter feedback by status
        tk.Label(self, text="Filter by Status").pack(pady=5)
        self.filter_var = tk.StringVar(value="All")
        self.filter_dropdown = tk.OptionMenu(self, self.filter_var, "All", "Integrated", "Not Integrated", command=self.filter_feedback)
        self.filter_dropdown.pack(pady=5)

        # Listbox to display feedback based on filter
        self.feedback_listbox = tk.Listbox(self, width=50, height=10)
        self.feedback_listbox.pack(pady=10)

        # Refresh button to reload latest CSV data
        refresh_button = tk.Button(self, text="Refresh Feedback", command=self.refresh_feedback)
        refresh_button.pack(pady=5)

        # Back button to return to the main menu (agent view only)
        back_button = tk.Button(self, text="Back to Main Menu", command=lambda: controller.show_frame(AgentMainMenu))
        back_button.pack(pady=5)

    def refresh_feedback(self):
        """Reload feedback data from CSV and repopulate list based on current filter."""
        # Reload the data from CSV using load_data()
        self.controller.users, self.controller.feedback_data, self.controller.report_data = load_data()
        # Apply the current filter after refreshing
        self.populate_feedback_list(self.filter_var.get())

    def populate_feedback_list(self, filter_status="All"):
        """Display feedback entries filtered by status in the listbox for the logged-in agent."""
        self.feedback_listbox.delete(0, tk.END)  # Clear the listbox

        # Check if current_user is set
        if not self.controller.current_user:
            self.feedback_listbox.insert(tk.END, "No feedback available.")
            return
        
        # Get the current agent's name from controller
        current_agent = self.controller.current_user['name']
        
        # Filter feedback data for the current agent and selected status
        feedback_data = [
            fb for fb in self.controller.feedback_data
            if fb["agent"] == current_agent and (filter_status == "All" or fb["status"] == filter_status)
        ]

        # Display filtered feedback entries in the listbox
        if feedback_data:
            for feedback in feedback_data:
                feedback_text = f"{feedback['feedback_id']}: {feedback['feedback_text']} (Status: {feedback['status']})"
                self.feedback_listbox.insert(tk.END, feedback_text)
        else:
            self.feedback_listbox.insert(tk.END, "No feedback available.")

    def filter_feedback(self, *args):
        """Filter feedback based on the selected status in the dropdown menu for the logged-in agent."""
        selected_filter = self.filter_var.get()
        self.populate_feedback_list(selected_filter)

    def refresh_feedback_on_login(self):
        """Clear and refresh feedback for the logged-in agent upon accessing this screen."""
        self.populate_feedback_list(self.filter_var.get())  # Refresh based on current filter status

In [11]:
if __name__ == "__main__":
    app = App()
    app.mainloop()